In [1]:
from starlette.applications import Starlette
from starlette.websockets import WebSocketDisconnect
import uvicorn, asyncio, json
import nest_asyncio
nest_asyncio.apply()

In [3]:
state = {'web_client': False, 'python_client': False}
websocket_dict = {}

app = Starlette()

async def close_client(websocket, msg):
    await websocket.send_json({'event': 'close', 'msg': msg})
    await websocket.close()

async def send_python_status(websocket=None, status=None):
    if websocket is None:
        if state['web_client']: websocket = websocket_dict['web_client']
        else: return
    if status is None: status = 'Online' if state['python_client'] else 'Offline'
    await websocket.send_json({'event': 'pythonClientStatus', 'msg': status})

async def handle_web_client(websocket):
    while True:
        try:
            msg = await websocket.receive_json()
            if msg['event'] == 'getPythonClientStatus':
                await send_python_status(websocket)
            else:
                print(f'Invalid event: ({msg["event"]!r}).')
        except json.JSONDecodeError: print('Invalid JSON')
        except WebSocketDisconnect: break
        
async def handle_python_client(websocket):
    await send_python_status()
    while True:
        try:
            msg = await websocket.receive_json()
            await websocket_dict['web_client'].send_json(msg)
        except json.JSONDecodeError: print('Invalid JSON')
        except WebSocketDisconnect:
            await send_python_status(status='Offline')
            break

func_dict = {'web_client': handle_web_client, 'python_client': handle_python_client}

@app.websocket_route('/ws')
async def websocket_endpoint(websocket):
    await websocket.accept()
    msg = await websocket.receive_json()
    client_type = msg['client']
    if client_type not in state:
        await close_client(websocket, f'Invalid client_type ({client_type!r})')
        return
    if state[client_type]:
        await close_client(websocket, 'Client already connected')
        return
    
    state[client_type] = True
    websocket_dict[client_type] = websocket
    await func_dict[client_type](websocket)
    
    # Close socket
    state[client_type] = False
    websocket_dict.pop(client_type)
    await close_client(websocket, 'Finalized by server')

In [4]:
uvicorn.run(app, host='0.0.0.0', port=8000)

INFO: Started server process [21977]
INFO: Waiting for application startup.
INFO: Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)
INFO: ('127.0.0.1', 60924) - "WebSocket /ws" [accepted]
INFO: ('127.0.0.1', 60992) - "WebSocket /ws" [accepted]
ERROR: Exception in ASGI application
Traceback (most recent call last):
  File "/home/renato/anaconda3/envs/ml/lib/python3.7/site-packages/uvicorn/protocols/websockets/websockets_impl.py", line 152, in run_asgi
    result = await self.app(self.scope, self.asgi_receive, self.asgi_send)
  File "/home/renato/anaconda3/envs/ml/lib/python3.7/site-packages/starlette/applications.py", line 102, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/home/renato/anaconda3/envs/ml/lib/python3.7/site-packages/starlette/middleware/errors.py", line 143, in __call__
    await self.app(scope, receive, send)
  File "/home/renato/anaconda3/envs/ml/lib/python3.7/site-packages/starlette/exceptions.py", line 58, in __call__
    await 

INFO: ('127.0.0.1', 32944) - "WebSocket /ws" [accepted]
ERROR: Exception in ASGI application
Traceback (most recent call last):
  File "/home/renato/anaconda3/envs/ml/lib/python3.7/site-packages/uvicorn/protocols/websockets/websockets_impl.py", line 152, in run_asgi
    result = await self.app(self.scope, self.asgi_receive, self.asgi_send)
  File "/home/renato/anaconda3/envs/ml/lib/python3.7/site-packages/starlette/applications.py", line 102, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/home/renato/anaconda3/envs/ml/lib/python3.7/site-packages/starlette/middleware/errors.py", line 143, in __call__
    await self.app(scope, receive, send)
  File "/home/renato/anaconda3/envs/ml/lib/python3.7/site-packages/starlette/exceptions.py", line 58, in __call__
    await self.app(scope, receive, send)
  File "/home/renato/anaconda3/envs/ml/lib/python3.7/site-packages/starlette/routing.py", line 550, in __call__
    await route.handle(scope, receive, send)
  File "/hom

INFO: ('127.0.0.1', 33260) - "WebSocket /ws" [accepted]
ERROR: Exception in ASGI application
Traceback (most recent call last):
  File "/home/renato/anaconda3/envs/ml/lib/python3.7/site-packages/uvicorn/protocols/websockets/websockets_impl.py", line 152, in run_asgi
    result = await self.app(self.scope, self.asgi_receive, self.asgi_send)
  File "/home/renato/anaconda3/envs/ml/lib/python3.7/site-packages/starlette/applications.py", line 102, in __call__
    await self.middleware_stack(scope, receive, send)
  File "/home/renato/anaconda3/envs/ml/lib/python3.7/site-packages/starlette/middleware/errors.py", line 143, in __call__
    await self.app(scope, receive, send)
  File "/home/renato/anaconda3/envs/ml/lib/python3.7/site-packages/starlette/exceptions.py", line 58, in __call__
    await self.app(scope, receive, send)
  File "/home/renato/anaconda3/envs/ml/lib/python3.7/site-packages/starlette/routing.py", line 550, in __call__
    await route.handle(scope, receive, send)
  File "/hom

INFO: Shutting down
ERROR: Exception in ASGI application
Traceback (most recent call last):
  File "/home/renato/anaconda3/envs/ml/lib/python3.7/site-packages/websockets/protocol.py", line 674, in transfer_data
    message = yield from self.read_message()
  File "/home/renato/anaconda3/envs/ml/lib/python3.7/site-packages/websockets/protocol.py", line 742, in read_message
    frame = yield from self.read_data_frame(max_size=self.max_size)
  File "/home/renato/anaconda3/envs/ml/lib/python3.7/site-packages/websockets/protocol.py", line 815, in read_data_frame
    frame = yield from self.read_frame(max_size)
  File "/home/renato/anaconda3/envs/ml/lib/python3.7/site-packages/websockets/protocol.py", line 884, in read_frame
    extensions=self.extensions,
  File "/home/renato/anaconda3/envs/ml/lib/python3.7/site-packages/websockets/framing.py", line 99, in read
    data = yield from reader(2)
  File "/home/renato/anaconda3/envs/ml/lib/python3.7/asyncio/streams.py", line 677, in readexactly
 

In [5]:
from starlette.websockets import WebSocket

In [37]:
WebSocket.sen